In [1]:
import scipy
import scipy.sparse
import numpy
import os
import re
import multiprocessing
import subprocess
import pickle
import matplotlib.pyplot

import time
import statistics
import sklearn
import math

In [2]:
CSR_mat_full = scipy.sparse.load_npz('./testdata/sparse_matrix_mumbach_without_dangling_non_reassigned_chr1.npz')

In [3]:
#needed to load the variables
with open("./testdata/variables.pi","rb") as picklefile:
    frag_index,frag_prop,frag_amount,valid_chroms,chroms_offsets = pickle.load(picklefile)

In [4]:
RE_pairs=CSR_mat_full.diagonal()
SC_pairs=CSR_mat_full.diagonal(k=1)
sum(RE_pairs)

1842362.0

In [5]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.plot(RE_pairs)
ax.plot(SC_pairs)
fig.show()

In [20]:
#preparing data for distribution plots
distribution = [[] for i in range(4000)]
diagonal_off1 = CSR_mat_full.diagonal(k=1).tolist()+[0]
diagonal_off1f = [0] + CSR_mat_full.diagonal(k=1).tolist() 
diagonal=CSR_mat_full.diagonal(k=0).tolist()
diagonal = [sum(x) for x in zip(diagonal_off1, diagonal_off1f,diagonal)]
lengths=[]
for i in frag_prop:
    lengths.append(i[3])
       
for i in range(len(lengths)):
    try:
        distribution[lengths[i]].append(diagonal[i])
    except:
        continue

results = []        
for i in distribution:
    if i:
        results.append(statistics.mean(i))
    else:
        results.append(1000)
        
for i in range(len(lengths)):
    try:
        if diagonal[i] != 0:
            distribution[lengths[i]].append(diagonal[i])
    except:
        continue        

results2 = []        
for i in distribution:
    if i:
        results2.append(statistics.mean(i))
    else:
        results2.append(1000)     

In [21]:
#plotting avarage reads vs length
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.scatter(range(4000),results)
ax.scatter(range(4000),results2)
fig.show()

In [6]:
def movingaverage (values, window):
    weights = numpy.repeat(1.0, window)/window
    sma = numpy.convolve(values, weights, 'same')
    return sma
def moving_integration (values, window):
    weights = numpy.repeat(1.0, window)
    sma = numpy.convolve(values, weights, 'same')
    return sma

In [17]:
#preparing data for distribution plots
distribution = [[] for i in range(6000)]
diagonal=CSR_mat_full.diagonal(k=0).tolist()
diagonal_off1 = CSR_mat_full.diagonal(k=1).tolist()+[0]
diagonal_off1f = [0] + CSR_mat_full.diagonal(k=1).tolist() 
diagonal = [sum(x) for x in zip(diagonal_off1, diagonal_off1f,diagonal)]
window=5
diagonal = moving_integration(diagonal,window)
lengths = []
for i in frag_prop:
    lengths.append(i[3])
       
lengths = moving_integration(lengths,window).tolist()
lengths = list(map(int, lengths))
for i in range(len(lengths)):
    try:
        distribution[lengths[i]].append(diagonal[i])
    except:
        continue

results = []        
for i in distribution:
    if i:
        results.append(statistics.mean(i))
    else:
        results.append(1000)
        
for i in range(len(lengths)):
    try:
        if diagonal[i] != 0:
            distribution[lengths[i]].append(diagonal[i])
    except:
        continue        

results2 = []        
for i in distribution:
    if i:
        results2.append(statistics.mean(i))
    else:
        results2.append(1000)   

In [18]:
#plotting avarage reads vs length
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.scatter(range(6000),results)
ax.scatter(range(6000),results2)
fig.show()

In [7]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
res= numpy.percentile(diagonal,95)
ax.plot(diagonal)
ax.axhline(res)
ax.plot(CSR_mat_full.sum(axis=0).tolist()[0])
fig.show()

NameError: name 'diagonal' is not defined

In [23]:
%matplotlib qt
#dirt way to do the lorentz curve on the fragments
fig = matplotlib.pyplot.figure()
ax = fig.add_subplot(111)
ax.plot(numpy.array(sorted(diagonal)).cumsum()/numpy.array(diagonal).sum())
ax.plot(numpy.array(sorted(CSR_mat_full.sum(axis=0).tolist()[0])).cumsum()/CSR_mat_full.sum(),label="full")

In [8]:
diagonal=CSR_mat_full.diagonal(k=0).tolist()
diagonal_off1 = CSR_mat_full.diagonal(k=1).tolist()+[0]
diagonal_off1f = [0] + CSR_mat_full.diagonal(k=1).tolist() 
diagonal = [sum(x) for x in zip(diagonal_off1, diagonal_off1f,diagonal)]

In [4]:
CSR_mat_full_withDE = scipy.sparse.load_npz('./testdata/sparse_matrix_mumbach_non_reassigned_chr1.npz')

In [9]:
diagonal2=CSR_mat_full_withDE.diagonal(k=0).tolist()
diagonal_off12 = CSR_mat_full_withDE.diagonal(k=1).tolist()+[0]
diagonal_off1f2 = [0] + CSR_mat_full_withDE.diagonal(k=1).tolist() 
diagonal2 = [sum(x) for x in zip(diagonal_off12, diagonal_off1f2,diagonal2)]
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
res= numpy.median(moving_integration(diagonal2,5))
ax.plot(moving_integration(diagonal2,5))
res2= numpy.percentile(moving_integration(diagonal,5),97)
ax.axhline(res)
ax.axhline(40)
print(res)
print(numpy.mean(moving_integration(diagonal2,5)))
#ax.axhline(res2)
#ax.plot(moving_integration(diagonal,5))
fig.show()

25.0
62.199282662461385


In [11]:
ma_diagonal=movingaverage(diagonal,5)
ma_diagonal2 = movingaverage(diagonal2,5)
sortedlist = sorted(range(len(ma_diagonal)), key=ma_diagonal.__getitem__, reverse=True)

In [24]:
%matplotlib qt
from ipywidgets import *
fig = matplotlib.pyplot.figure()
ax = fig.add_subplot(111)
matplotlib.pyplot.ion()
def interactplot(i):
    ax.clear()
    ax.plot(ma_diagonal2[sortedlist[i]-1500:sortedlist[i]+1500])
    bait = movingaverage(CSR_mat_full_withDE[sortedlist[i]-8:sortedlist[i]+8,].toarray().sum(axis=0).tolist()[sortedlist[i]-1500:sortedlist[i]+1500],5)
    ax.plot(bait)
    ratio = []
    for x, y in zip(bait, ma_diagonal[sortedlist[i]-1500:sortedlist[i]+1500]):
        if y > 10:
            ratio.append(x*100/(y+5))
        else:
            ratio.append(0)
    ratio2 = []
    for x, y in zip(bait, ma_diagonal2[sortedlist[i]-1500:sortedlist[i]+1500]):
        if y > 10:
            ratio2.append(x*100/(y+5))
        else:
            ratio2.append(-10)
    #ax.plot(ratio)
    #ax.plot(ratio2)
    fig.canvas.draw()
    print(sortedlist[i])
    print(frag_prop[sortedlist[i]])
    print(frag_prop[sortedlist[i]-1500])
    print(frag_prop[sortedlist[i]+1500])
    print(frag_prop[sortedlist[i]+1500][2]-frag_prop[sortedlist[i]-1500][1])
interact(interactplot,i=widgets.IntSlider(min=0,max=500,step=1,value=1));

#matplotlib.pyplot.plot(CSR_mat_full[index_max-5:index_max+5,].toarray().sum(axis=0).tolist()[index_max-200:index_max+200])

interactive(children=(IntSlider(value=1, description='i', max=500), Output()), _dom_classes=('widget-interact'…

In [21]:
fig = matplotlib.pyplot.figure()
ax = fig.add_subplot(111)
matplotlib.pyplot.ion()
def interactplot(i):
    ax.clear()
    ax.scatter(ma_diagonal2[sortedlist[i]-1500:sortedlist[i]+1500],movingaverage(CSR_mat_full_withDE[sortedlist[i]-8:sortedlist[i]+8,].toarray().sum(axis=0).tolist()[sortedlist[i]-1500:sortedlist[i]+1500],5),alpha=0.10)
    fig.canvas.draw()
interact(interactplot,i=widgets.IntSlider(min=0,max=500,step=1,value=1));

interactive(children=(IntSlider(value=1, description='i', max=500), Output()), _dom_classes=('widget-interact'…

In [22]:
fig = matplotlib.pyplot.figure()
ax = fig.add_subplot(111)
matplotlib.pyplot.ion()
def interactplot(i):
    ax.clear()
    ax.scatter(ma_diagonal2[sortedlist[i]-1000:sortedlist[i]-100],movingaverage(CSR_mat_full_withDE[sortedlist[i]-8:sortedlist[i]+8,].toarray().sum(axis=0).tolist()[sortedlist[i]-1000:sortedlist[i]-100],5),alpha=0.10)
    fig.canvas.draw()
interact(interactplot,i=widgets.IntSlider(min=0,max=500,step=1,value=1));

interactive(children=(IntSlider(value=1, description='i', max=500), Output()), _dom_classes=('widget-interact'…

In [14]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.plot(diagonal)
fig.show()

In [13]:
lengths=[]
distribution = [[] for i in range(6000)]
for i in frag_prop:
    lengths.append(i[3])
       
for i in range(len(lengths)):
    try:
        distribution[lengths[i]].append(diagonal[i])
    except:
        continue


In [14]:
lengths= lengths[:584662]

In [17]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.scatter(lengths,diagonal,alpha=0.01)
fig.show()

In [16]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.hist(lengths,range=(0,1000), bins = 1000)
fig.show()

ValueError: x must be strictly increasing